In [13]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml

class PerceptronReinforcement:
    def __init__(self, n_epochs=100, threshold=0.0, random_state=None):
        self.n_epochs = n_epochs
        self.threshold = threshold
        self.weights = None
        self.random_state = random_state
        self.converged = False

        if random_state is not None:
            np.random.seed(random_state)

    def _unit_step_function(self, x):
        return np.where(x >= self.threshold, 1, 0)

    def predict(self, X):
        linear_output = np.dot(X, self.weights)
        predictions = self._unit_step_function(linear_output)
        return predictions

    def train(self, X, y):
        n_samples, n_features = X.shape

        # Инициализация весов случайными малыми значениями
        self.weights = np.random.uniform(-0.1, 0.1, size=n_features)

        # Цикл обучения по эпохам
        for epoch in range(self.n_epochs):
            # Переменная для хранения количества ошибок в текущей эпохе
            epoch_error = 0

            # Цикл по обучающим векторам
            for idx, x in enumerate(X):
                # Подача входного образа
                Xm = x

                # Вычисление взвешенной суммы и выходного сигнала
                net = np.dot(Xm, self.weights)
                yj = self._unit_step_function(net)

                # Проверка соответствия выхода желаемому
                if yj != y[idx]:
                    # Увеличение счетчика ошибок
                    epoch_error += 1

                    # Увеличение весов активных входов (если выход = 0, а должен быть 1)
                    if yj == 0 and y[idx] == 1:
                        self.weights[Xm == 1] += 1 # Увеличение только для активных входов (xi == 1)

                    # Уменьшение весов активных входов (если выход = 1, а должен быть 0)
                    elif yj == 1 and y[idx] == 0:
                        self.weights[Xm == 1] -= 1 # Уменьшение только для активных входов (xi == 1)

            # Вывод ошибки за эпоху
            print(f"Эпоха {epoch + 1}, Ошибок: {epoch_error}/{n_samples}")

            # Проверка сходимости после каждой эпохи (если ошибок нет, то сходимость достигнута)
            if epoch_error == 0:
                self.converged = True
                print(f"Обучение завершено на эпохе {epoch+1} из-за сходимости.")
                break


            #Проверка на максимальное количество эпох
            if epoch == self.n_epochs - 1:
                print(f"Обучение завершено по достижении максимального количества эпох ({self.n_epochs}).")



def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)


def load_and_prepare_mnist(test_size=0.2, random_state=42, standardize=False, binary_classification=True, target_digit=0):
    try:
        mnist = fetch_openml('mnist_784', version=1, data_home=None, as_frame=False)
        X, y = mnist["data"], mnist["target"]
        y = y.astype(np.uint8)  # Преобразуем метки в целые числа
    except Exception as e:
        print(f"Ошибка при загрузке MNIST: {e}")
        return None, None, None, None


    if binary_classification:
        y = (y == target_digit).astype(int)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    if standardize:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        print("Предупреждение: Стандартизация данных может негативно повлиять на алгоритм обучения положительного и отрицательного подкрепления.")


    return X_train, X_test, y_train, y_test



if __name__ == '__main__':
    # 1. Загрузка и подготовка данных MNIST
    target_digit = 1  # Определяем, какую цифру будем распознавать (например, 0)
    X_train, X_test, y_train, y_test = load_and_prepare_mnist(target_digit=target_digit, standardize=False)

    if X_train is None:
        print("Ошибка при загрузке данных MNIST. Завершение программы.")
        exit()

    # 2. Инициализация и обучение персептрона
    perceptron = PerceptronReinforcement(n_epochs=20, threshold=0.0, random_state=101)
    perceptron.train(X_train, y_train)

    # 3. Оценка модели
    y_pred = perceptron.predict(X_test)
    acc = accuracy(y_test, y_pred)
    print(f"Точность: {acc}")


Эпоха 1, Ошибок: 8791/56000
Эпоха 2, Ошибок: 8656/56000
Эпоха 3, Ошибок: 8704/56000
Эпоха 4, Ошибок: 8622/56000
Эпоха 5, Ошибок: 8589/56000
Эпоха 6, Ошибок: 8607/56000
Эпоха 7, Ошибок: 8609/56000
Эпоха 8, Ошибок: 8619/56000
Эпоха 9, Ошибок: 8618/56000
Эпоха 10, Ошибок: 8619/56000
Эпоха 11, Ошибок: 8622/56000
Эпоха 12, Ошибок: 8617/56000
Эпоха 13, Ошибок: 8611/56000
Эпоха 14, Ошибок: 8609/56000
Эпоха 15, Ошибок: 8604/56000
Эпоха 16, Ошибок: 8602/56000
Эпоха 17, Ошибок: 8608/56000
Эпоха 18, Ошибок: 8613/56000
Эпоха 19, Ошибок: 8621/56000
Эпоха 20, Ошибок: 8619/56000
Обучение завершено по достижении максимального количества эпох (20).
Точность: 0.8446428571428571
